In [21]:
f= read("D:/federalistpapers.txt", String)

subEssay = String[]

i = 1
while i != nothing 
    next_occurrence_start = findnext("FEDERALIST No.", f, i)
    next_occurrence_end = findnext("PUBLIUS", f, i)
    println(SubString(f, next_occurrence_start[1], next_occurrence_end[2]))
    if next_occurrence_start == nothing
        break
    end
    i = next_occurrence_end[1] + 1
end


SystemError: SystemError: opening file "D:/federalistpapers.txt": No such file or directory

In [105]:
# TU PHAM
text = read("C:/Users/ADMIN/OneDrive/Documents/GitHub/csds133-s23-2/federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    processingEssay = essayParts[2]
    essays[i] = string(processingEssay[2:end], "PUBLIUS")
end

array = Array{String}[]
for i in eachindex(essays)
    paragraphs = split(essays[i], "\r\n\r\n")
    popfirst!(paragraphs)
    for j in eachindex(paragraphs)
        paragraphs[j] = replace(paragraphs[j], "\r\n" => "")
    end
    push!(array, paragraphs)
end
array

85-element Vector{Array{String}}:
 ["AFTER an unequivocal experience of the inefficacy of the subsistingfederal government, you are called upon to deliberate on a newConstitution for the United States of America. The subject speaks itsown importance; comprehending in its consequences nothing less than theexistence of the UNION, the safety and welfare of the parts of which itis composed, the fate of an empire in many respects the most interestingin the world. It has been frequently remarked that it seems to have beenreserved to the people of this country, by their conduct and example,to decide the important question, whether societies of men are reallycapable or not of establishing good government from reflection andchoice, or whether they are forever destined to depend for theirpolitical constitutions on accident and force. If there be any truthin the remark, the crisis at which we are arrived may with propriety beregarded as the era in which that decision is to be made; and a wrongele